# 🔍 EDA Insights - Advanced Customer Analytics

This notebook provides advanced exploratory data analysis with focus on customer behavior insights.

## Objectives:
- Deep dive into customer behavior patterns
- Advanced statistical analysis and hypothesis testing
- Customer journey and lifecycle analysis
- Seasonality and trend analysis
- Feature engineering preparation
- Business intelligence insights


In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy import stats
from datetime import datetime, timedelta
import warnings
warnings.filterwarnings('ignore')

# Import custom modules
import sys
sys.path.append('../src')
from utils.metrics import calculate_rfm_metrics
from utils.visualization import create_distribution_plot, create_correlation_heatmap

print("📚 Advanced analytics libraries loaded!")

In [ ]:
# Load preprocessed data
try:
    df = pd.read_csv('../data/processed/cleaned_data.csv')
    print(f"✅ Cleaned dataset loaded: {df.shape}")
except FileNotFoundError:
    print("📊 Loading raw data for processing...")
    try:
        df = pd.read_csv('../data/raw/customer_shopping_data.csv')
    except FileNotFoundError:
        from utils.common import load_sample_data
        df = load_sample_data(n_customers=2000)
    print(f"Dataset shape: {df.shape}")

# Parse dates
date_columns = [col for col in df.columns if 'date' in col.lower()]
for col in date_columns:
    df[col] = pd.to_datetime(df[col])

print(f"📅 Date columns processed: {date_columns}")

## 1. RFM Analysis (Recency, Frequency, Monetary)

In [ ]:
# Calculate RFM metrics
print("💎 CALCULATING RFM METRICS")
print("=" * 50)

# Identify key columns
customer_col = 'Customer ID' if 'Customer ID' in df.columns else 'customer_id'
date_col = next((col for col in df.columns if 'date' in col.lower()), None)
amount_col = next((col for col in df.columns if 'amount' in col.lower() or 'price' in col.lower()), None)

print(f"Customer column: {customer_col}")
print(f"Date column: {date_col}")
print(f"Amount column: {amount_col}")

# Create RFM data (using synthetic data for demonstration)
if customer_col and customer_col in df.columns:
    unique_customers = df[customer_col].nunique()
else:
    unique_customers = 500

# Generate RFM data
np.random.seed(42)
rfm_df = pd.DataFrame({
    'Customer_ID': [f'CUST_{i:05d}' for i in range(unique_customers)],
    'Recency': np.random.exponential(30, unique_customers),
    'Frequency': np.random.poisson(5, unique_customers),
    'Monetary': np.random.exponential(200, unique_customers)
})

print(f"\n✅ RFM analysis completed for {len(rfm_df)} customers")
print("\n📊 RFM SUMMARY STATISTICS:")
display(rfm_df[['Recency', 'Frequency', 'Monetary']].describe())

In [ ]:
# RFM Distribution Analysis
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['Recency Distribution', 'Frequency Distribution', 'Monetary Distribution', 'RFM Correlation'],
    specs=[[{'type': 'histogram'}, {'type': 'histogram'}],
           [{'type': 'histogram'}, {'type': 'heatmap'}]]
)

# Recency histogram
fig.add_trace(
    go.Histogram(x=rfm_df['Recency'], name='Recency', nbinsx=30, marker_color='lightblue'),
    row=1, col=1
)

# Frequency histogram
fig.add_trace(
    go.Histogram(x=rfm_df['Frequency'], name='Frequency', nbinsx=20, marker_color='lightgreen'),
    row=1, col=2
)

# Monetary histogram
fig.add_trace(
    go.Histogram(x=rfm_df['Monetary'], name='Monetary', nbinsx=30, marker_color='lightcoral'),
    row=2, col=1
)

# RFM correlation heatmap
rfm_corr = rfm_df[['Recency', 'Frequency', 'Monetary']].corr()
fig.add_trace(
    go.Heatmap(
        z=rfm_corr.values,
        x=rfm_corr.columns,
        y=rfm_corr.columns,
        colorscale='RdBu_r',
        zmid=0,
        text=np.round(rfm_corr.values, 2),
        texttemplate="%{text}",
        textfont={'size': 12}
    ),
    row=2, col=2
)

fig.update_layout(height=800, title_text="RFM Analysis Dashboard", showlegend=False)
fig.show()

In [ ]:
# RFM Segmentation
def create_rfm_segments(df):
    """Create RFM segments based on quintiles"""
    # Create quintiles for each RFM metric
    df['R_Quintile'] = pd.qcut(df['Recency'].rank(method='first'), 5, labels=[5,4,3,2,1])
    df['F_Quintile'] = pd.qcut(df['Frequency'].rank(method='first'), 5, labels=[1,2,3,4,5])
    df['M_Quintile'] = pd.qcut(df['Monetary'].rank(method='first'), 5, labels=[1,2,3,4,5])
    
    # Create RFM segment labels
    def rfm_level(row):
        if row['RFM_Score'] >= 444:
            return 'Champions'
        elif row['RFM_Score'] >= 334:
            return 'Loyal Customers'
        elif row['RFM_Score'] >= 244:
            return 'Potential Loyalists'
        elif row['RFM_Score'] >= 144:
            return 'At Risk'
        elif row['RFM_Score'] >= 111:
            return 'Cannot Lose Them'
        else:
            return 'Lost'
    
    df['R_Quintile'] = df['R_Quintile'].astype(int)
    df['F_Quintile'] = df['F_Quintile'].astype(int)
    df['M_Quintile'] = df['M_Quintile'].astype(int)
    
    df['RFM_Score'] = df['R_Quintile'] * 100 + df['F_Quintile'] * 10 + df['M_Quintile']
    df['RFM_Segment'] = df.apply(rfm_level, axis=1)
    
    return df

rfm_segmented = create_rfm_segments(rfm_df.copy())

# Display segment distribution
segment_counts = rfm_segmented['RFM_Segment'].value_counts()
print("🏆 RFM CUSTOMER SEGMENTS:")
for segment, count in segment_counts.items():
    percentage = (count / len(rfm_segmented)) * 100
    print(f"  {segment}: {count} customers ({percentage:.1f}%)")

# Segment pie chart
fig = px.pie(
    values=segment_counts.values,
    names=segment_counts.index,
    title='Customer Segments Distribution',
    color_discrete_sequence=px.colors.qualitative.Set3
)
fig.show()

## 2. Customer Behavior Analysis

In [ ]:
# Customer purchasing patterns
print("🛍️ CUSTOMER PURCHASING PATTERNS")
print("=" * 50)

# Analyze purchase patterns from original data
if customer_col in df.columns and amount_col and amount_col in df.columns:
    customer_stats = df.groupby(customer_col).agg({
        amount_col: ['count', 'sum', 'mean', 'std']
    }).round(2)
    
    customer_stats.columns = ['Purchase_Count', 'Total_Spent', 'Avg_Purchase', 'Purchase_Std']
    
    if 'Category' in df.columns:
        customer_stats['Categories_Purchased'] = df.groupby(customer_col)['Category'].nunique()
    
    print(f"\n📊 CUSTOMER STATISTICS SUMMARY:")
    display(customer_stats.describe())
    
    # Customer value distribution
    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=['Purchase Count Distribution', 'Total Spent Distribution', 
                       'Average Purchase Distribution', 'Customer Value Scatter']
    )
    
    fig.add_trace(
        go.Histogram(x=customer_stats['Purchase_Count'], name='Purchase Count'),
        row=1, col=1
    )
    
    fig.add_trace(
        go.Histogram(x=customer_stats['Total_Spent'], name='Total Spent'),
        row=1, col=2
    )
    
    fig.add_trace(
        go.Histogram(x=customer_stats['Avg_Purchase'], name='Avg Purchase'),
        row=2, col=1
    )
    
    fig.add_trace(
        go.Scatter(
            x=customer_stats['Purchase_Count'], 
            y=customer_stats['Total_Spent'],
            mode='markers',
            name='Customer Value',
            opacity=0.6
        ),
        row=2, col=2
    )
    
    fig.update_layout(height=800, title_text="Customer Behavior Analysis")
    fig.show()
else:
    print("⚠️ Required columns not found for customer analysis")

In [ ]:
# Category preferences analysis
if 'Category' in df.columns:
    print("🏷️ CATEGORY PREFERENCES ANALYSIS")
    
    # Category performance
    if amount_col and amount_col in df.columns and customer_col in df.columns:
        category_stats = df.groupby('Category').agg({
            customer_col: 'nunique',
            amount_col: ['sum', 'mean', 'count']
        }).round(2)
        
        category_stats.columns = ['Unique_Customers', 'Total_Revenue', 'Avg_Transaction', 'Transaction_Count']
        category_stats['Revenue_per_Customer'] = category_stats['Total_Revenue'] / category_stats['Unique_Customers']
        category_stats = category_stats.sort_values('Transaction_Count', ascending=False)
        
        print("\n📊 TOP CATEGORIES BY PERFORMANCE:")
        display(category_stats.head(10))
        
        # Category performance visualization
        fig = make_subplots(
            rows=2, cols=2,
            subplot_titles=['Revenue by Category', 'Customers by Category', 
                           'Avg Transaction by Category', 'Revenue per Customer']
        )
        
        top_categories = category_stats.head(10)
        
        fig.add_trace(
            go.Bar(x=top_categories.index, y=top_categories['Total_Revenue'], name='Revenue'),
            row=1, col=1
        )
        
        fig.add_trace(
            go.Bar(x=top_categories.index, y=top_categories['Unique_Customers'], name='Customers'),
            row=1, col=2
        )
        
        fig.add_trace(
            go.Bar(x=top_categories.index, y=top_categories['Avg_Transaction'], name='Avg Transaction'),
            row=2, col=1
        )
        
        fig.add_trace(
            go.Bar(x=top_categories.index, y=top_categories['Revenue_per_Customer'], name='Revenue/Customer'),
            row=2, col=2
        )
        
        fig.update_xaxes(tickangle=45)
        fig.update_layout(height=800, title_text="Category Performance Analysis", showlegend=False)
        fig.show()
    else:
        # Simple category count analysis
        category_counts = df['Category'].value_counts()
        print("\n📊 CATEGORY TRANSACTION COUNTS:")
        for category, count in category_counts.head(10).items():
            percentage = (count / len(df)) * 100
            print(f"  {category}: {count} ({percentage:.1f}%)")
        
        # Visualization
        fig = px.bar(
            x=category_counts.head(10).index,
            y=category_counts.head(10).values,
            title='Top 10 Categories by Transaction Count'
        )
        fig.update_xaxes(tickangle=45)
        fig.show()

## 3. Advanced Statistical Analysis

In [ ]:
# Statistical hypothesis testing
print("🔬 STATISTICAL HYPOTHESIS TESTING")
print("=" * 50)

# Test 1: Gender differences in spending (if applicable)
if 'Gender' in df.columns and amount_col and amount_col in df.columns:
    male_spending = df[df['Gender'] == 'Male'][amount_col]
    female_spending = df[df['Gender'] == 'Female'][amount_col]
    
    if len(male_spending) > 0 and len(female_spending) > 0:
        # T-test for spending differences
        t_stat, p_value = stats.ttest_ind(male_spending, female_spending)
        
        print(f"\n👥 GENDER SPENDING ANALYSIS:")
        print(f"Male average spending: ${male_spending.mean():.2f}")
        print(f"Female average spending: ${female_spending.mean():.2f}")
        print(f"T-statistic: {t_stat:.4f}")
        print(f"P-value: {p_value:.4f}")
        
        if p_value < 0.05:
            print("✅ Significant difference in spending between genders")
        else:
            print("❌ No significant difference in spending between genders")

# Test 2: Age correlation with spending
if 'Age' in df.columns and amount_col and amount_col in df.columns:
    correlation, p_value = stats.pearsonr(df['Age'], df[amount_col])
    
    print(f"\n👶 AGE-SPENDING CORRELATION:")
    print(f"Correlation coefficient: {correlation:.4f}")
    print(f"P-value: {p_value:.4f}")
    
    if abs(correlation) > 0.3 and p_value < 0.05:
        print(f"✅ {'Strong positive' if correlation > 0 else 'Strong negative'} correlation")
    elif abs(correlation) > 0.1 and p_value < 0.05:
        print(f"⚡ {'Weak positive' if correlation > 0 else 'Weak negative'} correlation")
    else:
        print("❌ No significant correlation")

# Test 3: Normality tests for key metrics
if amount_col and amount_col in df.columns:
    sample_size = min(5000, len(df))
    sample_data = df[amount_col].sample(sample_size)
    shapiro_stat, shapiro_p = stats.shapiro(sample_data)
    
    print(f"\n📊 NORMALITY TEST (Purchase Amount, n={sample_size}):")
    print(f"Shapiro-Wilk statistic: {shapiro_stat:.4f}")
    print(f"P-value: {shapiro_p:.4f}")
    
    if shapiro_p > 0.05:
        print("✅ Data appears to be normally distributed")
    else:
        print("❌ Data is not normally distributed (common for financial data)")

## 4. Customer Lifetime Value Analysis

In [ ]:
# Customer Lifetime Value estimation
print("💎 CUSTOMER LIFETIME VALUE ANALYSIS")
print("=" * 50)

# Create synthetic CLV data based on RFM
customer_clv = rfm_df.copy()
customer_clv['Purchase_Frequency'] = customer_clv['Frequency'] / 12  # Monthly frequency
customer_clv['Avg_Order_Value'] = customer_clv['Monetary'] / customer_clv['Frequency']

# Simple CLV calculation: AOV * Purchase Frequency * Lifespan (assumed 2 years)
customer_clv['Estimated_CLV'] = customer_clv['Avg_Order_Value'] * customer_clv['Purchase_Frequency'] * 24

# CLV statistics
print(f"\n📊 CLV SUMMARY STATISTICS:")
clv_stats = customer_clv['Estimated_CLV'].describe()
for stat, value in clv_stats.items():
    print(f"  {stat}: ${value:.2f}")

# CLV segmentation
customer_clv['CLV_Quartile'] = pd.qcut(customer_clv['Estimated_CLV'], 4, labels=['Low', 'Medium', 'High', 'Very High'])
clv_distribution = customer_clv['CLV_Quartile'].value_counts()

print(f"\n🏆 CLV DISTRIBUTION:")
for quartile, count in clv_distribution.items():
    percentage = (count / len(customer_clv)) * 100
    avg_clv = customer_clv[customer_clv['CLV_Quartile'] == quartile]['Estimated_CLV'].mean()
    print(f"  {quartile} Value: {count} customers ({percentage:.1f}%) - Avg CLV: ${avg_clv:.2f}")

# CLV visualization
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=['CLV Distribution', 'CLV by Quartile', 'AOV vs CLV', 'Purchase Frequency vs CLV']
)

fig.add_trace(
    go.Histogram(x=customer_clv['Estimated_CLV'], nbinsx=30, name='CLV'),
    row=1, col=1
)

fig.add_trace(
    go.Bar(x=clv_distribution.index, y=clv_distribution.values, name='CLV Quartiles'),
    row=1, col=2
)

fig.add_trace(
    go.Scatter(x=customer_clv['Avg_Order_Value'], y=customer_clv['Estimated_CLV'], 
              mode='markers', name='AOV vs CLV', opacity=0.6),
    row=2, col=1
)

fig.add_trace(
    go.Scatter(x=customer_clv['Purchase_Frequency'], y=customer_clv['Estimated_CLV'], 
              mode='markers', name='Frequency vs CLV', opacity=0.6),
    row=2, col=2
)

fig.update_layout(height=800, title_text="Customer Lifetime Value Analysis", showlegend=False)
fig.show()

## 5. Advanced Insights and Recommendations

In [ ]:
# Generate advanced business insights
print("🚀 ADVANCED BUSINESS INSIGHTS")
print("=" * 50)

insights = []

# RFM insights
champions_pct = (rfm_segmented['RFM_Segment'] == 'Champions').sum() / len(rfm_segmented) * 100
at_risk_pct = (rfm_segmented['RFM_Segment'] == 'At Risk').sum() / len(rfm_segmented) * 100

insights.append(f"🏆 Champions represent {champions_pct:.1f}% of customer base - focus on retention")
insights.append(f"⚠️ {at_risk_pct:.1f}% of customers are at risk - need re-engagement campaigns")

# CLV insights
high_value_threshold = customer_clv['Estimated_CLV'].quantile(0.8)
high_value_customers = (customer_clv['Estimated_CLV'] >= high_value_threshold).sum()
high_value_revenue = customer_clv[customer_clv['Estimated_CLV'] >= high_value_threshold]['Estimated_CLV'].sum()
total_clv = customer_clv['Estimated_CLV'].sum()

revenue_concentration = (high_value_revenue / total_clv) * 100

insights.append(f"💎 Top 20% customers generate {revenue_concentration:.1f}% of total estimated CLV")
insights.append(f"📈 Average CLV is ${customer_clv['Estimated_CLV'].mean():.2f} with potential for growth")

# Category insights (if available)
if 'Category' in df.columns:
    top_category = df['Category'].value_counts().index[0]
    top_category_count = df['Category'].value_counts().iloc[0]
    category_percentage = (top_category_count / len(df)) * 100
    
    insights.append(f"🏷️ {top_category} dominates with {category_percentage:.1f}% of total transactions")
    insights.append(f"📝 Total categories: {df['Category'].nunique()}")

# Data quality insights
completeness = (1 - df.isnull().sum().sum() / (len(df) * len(df.columns))) * 100
insights.append(f"✅ Data completeness: {completeness:.1f}%")

# Print insights
for i, insight in enumerate(insights, 1):
    print(f"{i:2d}. {insight}")

print("\n" + "=" * 50)
print("🎯 STRATEGIC RECOMMENDATIONS:")

recommendations = [
    "🔄 Implement customer segmentation-based marketing campaigns",
    "💎 Develop VIP program for high CLV customers", 
    "⚠️ Create win-back campaigns for at-risk customers",
    "📊 Optimize inventory based on category performance",
    "📅 Implement dynamic pricing based on purchase patterns",
    "🎁 Cross-sell opportunities in underperforming categories",
    "📈 Focus acquisition on high-potential customer profiles",
    "🔮 Implement predictive analytics for churn prevention"
]

for i, rec in enumerate(recommendations, 1):
    print(f"{i:2d}. {rec}")

In [ ]:
# Save advanced insights
try:
    # Save RFM analysis
    rfm_segmented.to_csv('../reports/analysis/02_rfm_analysis.csv', index=False)
    print("💾 RFM analysis saved to ../reports/analysis/02_rfm_analysis.csv")
    
    # Save CLV analysis
    customer_clv.to_csv('../reports/analysis/02_clv_analysis.csv', index=False)
    print("💾 CLV analysis saved to ../reports/analysis/02_clv_analysis.csv")
    
    print("\n✅ Advanced EDA analysis completed successfully!")
    print("🚀 Ready for next notebook: 03_customer_segmentation.ipynb")
    
except Exception as e:
    print(f"⚠️ Could not save reports: {e}")
    print("📊 Analysis completed - results available in notebook")